In [ ]:
import time
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [ ]:
# 目的是将一个函数 添加为 指定类的方法
def add_to_class(Class):  #@save
    """Register functions as methods in created class."""
    def wrapper(obj):
       # 给Class类添加新方法（类，方法名，方法）
        setattr(Class, obj.__name__, obj)
    return wrapper


'''eg:
class MyClass:
    pass

@add_to_class(MyClass)
def foo(self, x):
    print(x)

# 现在，foo成为了MyClass的一个方法
instance = MyClass()
instance.foo("Hello")  # 输出: Hello

'''

In [ ]:
class HyperParameters: #@save
  '''The base class of hyperparameters'''
  def save_hyperparameter(self, ignore=[]):
    raise NotImplemented

In [ ]:
# Call the fully implemented HyperParameters class saved in d2l
class B(d2l.HyperParameters):
    def __init__(self, a, b, c):
        self.save_hyperparameters(ignore=['c'])
        print('self.a =', self.a, 'self.b =', self.b)
        print('There is no self.c =', not hasattr(self, 'c'))

b = B(a=1, b=2, c=3)

In [ ]:
# 使用场景这种方法使得在类的方法中调用save_hyperparameters时能够自动将该方法的参数转换为类实例的属性，
# 非常适合于那些有大量配置项（如训练超参数）需要管理的情况。
# 这样，你可以在类的初始化方法__init__中调用save_hyperparameters，自动保存所有传入的初始化参数为类的属性，便于后续访问和管理。
class HyperParameters:
    """The base class of hyperparameters."""
    def save_hyperparameters(self, ignore=[]):
        """Defined in :numref:`sec_oo-design`"""
        raise NotImplemented

    def save_hyperparameters(self, ignore=[]):
        """Save function arguments into class attributes.
    
        Defined in :numref:`sec_utils`"""
        #获取调用栈：通过inspect.currentframe().f_back获取当前函数的调用者的帧对象。
        # 这使得方法能够访问调用它的函数的局部变量。
        frame = inspect.currentframe().f_back
        # 解析局部变量：inspect.getargvalues(frame)用于获取调用者的局部变量，返回一个包含四个元素的元组，
        # 其中local_vars是我们感兴趣的部分，它包含了调用者的局部变量和对应的值。
        _, _, _, local_vars = inspect.getargvalues(frame)
        # 过滤并保存参数：通过列表推导式和条件过滤，排除掉ignore列表中指定的变量名、以及名称中以self或下划线开头的变量。
        # 然后，将剩余的变量保存到实例的hparams字典属性中。这些变量同样被设置为类实例的属性，使得可以通过self.variable_name的方式直接访问这些参数。
        self.hparams = {k:v for k, v in local_vars.items()
                        if k not in set(ignore+['self']) and not k.startswith('_')}
        for k, v in self.hparams.items():
            setattr(self, k, v)
            
'''
import inspect

class MyModel(HyperParameters):
    def __init__(self, learning_rate, num_layers):
        self.save_hyperparameters()

model = MyModel(0.01, 3)
print(model.hparams)  # 输出: {'learning_rate': 0.01, 'num_layers': 3}

'''

In [ ]:
class ProgressBoard(d2l.HyperParameters):  #@save
    """The board that plots data points in animation."""
    def __init__(self, 
                 xlabel=None, ylabel=None, # x y轴的标签
                 xlim=None,ylim=None, # x y轴的限制
                 xscale='linear', yscale='linear', # x y轴的缩放类型 还有‘log’类型
                 ls=['-', '--', '-.', ':'], # 线型列表，例如实线虚线
                 colors=['C0', 'C1', 'C2', 'C3'],
                 fig=None, axes=None, # 如果提供，可以在这些已有的图形和轴上进行绘图，否则将创建新的。
                 figsize=(3.5, 2.5), # 创建新图形时的尺寸。
                 display=True): # 是否显示图形
        self.save_hyperparameters()

    def draw(self, x, y, label, every_n=1):
        """Defined in :numref:`sec_utils`"""
        Point = collections.namedtuple('Point', ['x', 'y'])
        if not hasattr(self, 'raw_points'):
            self.raw_points = collections.OrderedDict()
            self.data = collections.OrderedDict()
        if label not in self.raw_points:
            self.raw_points[label] = []
            self.data[label] = []
        points = self.raw_points[label]
        line = self.data[label]
        points.append(Point(x, y))
        if len(points) != every_n:
            return
        mean = lambda x: sum(x) / len(x)
        line.append(Point(mean([p.x for p in points]),
                          mean([p.y for p in points])))
        points.clear()
        if not self.display:
            return
        d2l.use_svg_display()
        if self.fig is None:
            self.fig = d2l.plt.figure(figsize=self.figsize)
        plt_lines, labels = [], []
        for (k, v), ls, color in zip(self.data.items(), self.ls, self.colors):
            plt_lines.append(d2l.plt.plot([p.x for p in v], [p.y for p in v],
                                          linestyle=ls, color=color)[0])
            labels.append(k)
        axes = self.axes if self.axes else d2l.plt.gca()
        if self.xlim: axes.set_xlim(self.xlim)
        if self.ylim: axes.set_ylim(self.ylim)
        if not self.xlabel: self.xlabel = self.x
        axes.set_xlabel(self.xlabel)
        axes.set_ylabel(self.ylabel)
        axes.set_xscale(self.xscale)
        axes.set_yscale(self.yscale)
        axes.legend(plt_lines, labels)
        display.display(self.fig)
        display.clear_output(wait=True)

In [ ]:
board = d2l.ProgressBoard('x')
for x in np.arange(0, 10, 0.1): # 从0到10，步长为0.1
    board.draw(x, np.sin(x), 'sin', every_n=2)
    board.draw(x, np.cos(x), 'cos', every_n=10)

In [ ]:
class Module(nn.Module, d2l.HyperParameters):  #@save
    """The base class of models."""
    # 指定每个epoch训练和验证阶段的绘图频率
    def __init__(self, plot_train_per_epoch=2, plot_valid_per_epoch=1):
        super().__init__()
        self.save_hyperparameters()
        self.board = ProgressBoard()

    # 需要子类实现，用于计算模型的损失。
    def loss(self, y_hat, y):
        raise NotImplementedError

    # 定义了模型的前向传播逻辑。要求类中必须有net属性（即实际的神经网络模型），并使用这个网络对输入X进行处理。
    def forward(self, X):
        assert hasattr(self, 'net'), 'Neural network is defined'
        return self.net(X)

    # 根据传入的键值对和标志位（表示是训练还是验证阶段），在ProgressBoard上绘制动态图表
    def plot(self, key, value, train):
        """Plot a point in animation."""
        assert hasattr(self, 'trainer'), 'Trainer is not inited'
        self.board.xlabel = 'epoch'
        if train:
            x = self.trainer.train_batch_idx / \
                self.trainer.num_train_batches
            n = self.trainer.num_train_batches / \
                self.plot_train_per_epoch
        else:
            x = self.trainer.epoch + 1
            n = self.trainer.num_val_batches / \
                self.plot_valid_per_epoch
        self.board.draw(x, value.to(d2l.cpu()).detach().numpy(),
                        ('train_' if train else 'val_') + key,
                        every_n=int(n))

    # 定义了单个批次的训练步骤，包括计算损失和绘制训练损失。
    def training_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        self.plot('loss', l, train=True)
        return l

    # 定义了单个批次的验证步骤，包括计算损失和绘制验证损失。
    def validation_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        self.plot('loss', l, train=False)

    # 一个抽象方法，需要在子类中定义，用于配置模型的优化器。
    def configure_optimizers(self):
        raise NotImplementedError

In [ ]:
class DataModule(d2l.HyperParameters):  #@save
    """The base class of data."""
    def __init__(self, root='../data', num_workers=4):
        self.save_hyperparameters()

    def get_dataloader(self, train):
        raise NotImplementedError

    def train_dataloader(self):
        return self.get_dataloader(train=True)

    def val_dataloader(self):
        return self.get_dataloader(train=False)

In [ ]:
class Trainer(d2l.HyperParameters):  #@save
    """The base class for training models with data."""

    def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
        self.save_hyperparameters()
        assert num_gpus == 0, 'No GPU support yet'

    def prepare_data(self, data):
        self.train_dataloader = data.train_dataloader()
        self.val_dataloader = data.val_dataloader()
        self.num_train_batches = len(self.train_dataloader)
        self.num_val_batches = (len(self.val_dataloader)
                                if self.val_dataloader is not None else 0)

    def prepare_model(self, model:d2l.Module):
        model.trainer = self
        model.board.xlim = [0, self.max_epochs]
        self.model = model

    def fit(self, model, data):
        self.prepare_data(data)
        self.prepare_model(model)
        self.optim = model.configure_optimizers()
        self.epoch = 0
        self.train_batch_idx = 0
        self.val_batch_idx = 0
        for self.epoch in range(self.max_epochs):
            self.fit_epoch()

    # from 3.4
    def prepare_batch(self, batch):
        return batch

    # from 3.4
    def fit_epoch(self):
        self.model.train()
        for batch in self.train_dataloader:
            loss = self.model.training_step(self.prepare_batch(batch))
            self.optim.zero_grad()
            with torch.no_grad():
                loss.backward()
                if self.gradient_clip_val > 0:  # To be discussed later
                    self.clip_gradients(self.gradient_clip_val, self.model)
                self.optim.step()
            self.train_batch_idx += 1
        if self.val_dataloader is None:
            return
        self.model.eval()
        for batch in self.val_dataloader:
            with torch.no_grad():
                self.model.validation_step(self.prepare_batch(batch))
            self.val_batch_idx += 1